In [3]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


############################################
# 1. Chargement et préparation des données
############################################


df = pd.read_csv("../data/advertising_with_dummies/Dummy Data HSS.csv")


# Gestion des données manquantes
# Remplissage des valeurs manquantes par la moyenne de chaque colonne
df.fillna(df.mean(), inplace=True)

# Conversion des variables catégorielles en variables indicatrices
# Suppression de la première catégorie pour éviter la multicolinéarité
category_dummies = pd.get_dummies(df['Influencer'], drop_first=True)
df = pd.concat([df, category_dummies], axis=1)
df.drop(['Influencer'], axis=1, inplace=True)

# Séparation des variables indépendantes et de la variable cible
X = df.drop('Sales', axis=1)
y = df['Sales']


############################################
# 2. Séparation des données
############################################

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



############################################
# 3. Normalisation des données
############################################
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


############################################
# 4. Création et entraînement du modèle de régression linéaire
############################################

model = LinearRegression()
model.fit(X_train_scaled, y_train)


#########################
# PREDICTION
#########################

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test_scaled)


############################################
# 5. Évaluation du modèle
############################################


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R^2:", r2)


############################################
# 6. Validation croisée
############################################

# Utilisation de la validation croisée pour une évaluation plus robuste
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("CV average score: %.2f" % cv_scores.mean())



############################################
# 7. Optimisation des hyperparamètres
############################################

# Utilisation de GridSearchCV pour optimiser les hyperparamètres du modèle Ridge
parameters = {'alpha': [0.1, 0.5, 1.0, 10.0, 100.0]}
ridge = Ridge()
grid = GridSearchCV(ridge, parameters, cv=5)
grid.fit(X_train_scaled, y_train)



# Meilleur modèle et score
print("Best parameters:", grid.best_params_)
print("Best cross-validation score: %.2f" % grid.best_score_)

############################################
# 8. Évaluation du modèle optimisé¶
############################################


# Prédiction avec le modèle Ridge optimisé
best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)
print("Best model MSE:", best_mse)
print("Best model R^2:", best_r2)

MSE: 63.413990338050375
R^2: 0.9924019878236671
CV average score: 0.99
Best parameters: {'alpha': 0.1}
Best cross-validation score: 0.99
Best model MSE: 63.414579818235985
Best model R^2: 0.9924019171944951


/var/folders/yf/7zryq8v115v8hjpz3_522bvm0000gn/T/ipykernel_3458/3384027064.py:20: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)
